In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl

In [11]:
# protein = '1fme'
# md_t2 = {'1fme': 18*1000}
lag = 31
processes = 2


In [12]:
summary_1fme = '/Users/robertarbon/Library/CloudStorage/GoogleDrive-robert.arbon@gmail.com/My Drive/msm_optimisation/1fme/summary.h5'
summary_cln = '/Users/robertarbon/Library/CloudStorage/GoogleDrive-robert.arbon@gmail.com/My Drive/msm_optimisation/msm_opt/CLN/summary.h5'
hp_path = '/Users/robertarbon/Library/CloudStorage/GoogleDrive-robert.arbon@gmail.com/My Drive/msm_optimisation/hpsample.h5'

In [13]:

def get_timescales(hp_path, summary_path):

    hp = pd.read_hdf(hp_path)
    hp.reset_index(inplace=True)
    for drop_col in ['index', 'Group']:
        if drop_col in hp.columns:
            hp.drop(labels=[drop_col], inplace=True, axis=1)
    
    
    timescales = pd.read_hdf(summary_path, key='timescales')
    timescales.reset_index(inplace=True)
    for drop_col in ['index', 'Group']:
        if drop_col in timescales.columns:
            timescales.drop(labels=[drop_col], inplace=True, axis=1)
    
    timescales = timescales.merge(hp, on=['hp_ix'], how='left')
    timescales['feature'] = timescales.apply(lambda x: f"{x['feature__value']}" if x['feature__value'] =='dihedrals' else f"{x['distances__transform']}-{x['feature__value']}", axis=1)
    print(timescales.shape)
    timescales.drop_duplicates(inplace=True)
    print(timescales.shape)
    timescales = timescales.loc[timescales.hp_ix < 329, :].copy()

    data=timescales.query(f"process=={processes}").query(f'lag=={lag}')
    data.sort_values(by='median', inplace=True, axis=0, ascending=False)
    data['rank'] = (np.arange(data.shape[0])+1).astype(int)

    short_names = {'dihedrals': 'dihed.','logistic-distances': 'logit(dist.)', 'linear-distances': 
              'dist.'}

    def f(x):
        for short, long in short_names.items(): 
            x = x.replace(short, long)
        return x
    
    data['feature'] = data['feature'].apply(f)
    # timescales['feature'] = timescales['feature'].apply(f)
    return data


In [14]:
data_dict = {'BBA': get_timescales(hp_path, summary_1fme), 
        'CLN025': get_timescales(hp_path, summary_cln)}

(68711, 21)
(68711, 21)
(4979, 21)
(4979, 21)


In [15]:
data = data_dict['CLN025']

In [16]:
# data.sort_values(by='median', ascending=False).loc[:, ['median', 'lb', 'ub',  'hp_ix','distances__scheme', 'tica__dim', 'tica__lag', 'cluster__k', 'distances__steepness', 'distances__centre']].head(12)

In [22]:
data.query('hp_ix==74').loc[:,  ['median', 'lb', 'ub',  'hp_ix','distances__scheme', 
                                       'tica__dim', 'tica__lag', 'cluster__k', 'distances__steepness', 
                                       'distances__centre']]

,median,lb,ub,hp_ix,distances__scheme,tica__dim,tica__lag,cluster__k,distances__steepness,distances__centre
236,379.427824,296.450766,469.664699,74,ca,15,71,488,0.0,0.0


In [26]:
def label(x): 
    # msg = f"\SI{{{x['median']/1000:3.1f}}}{{\{'micro'}\{'second'}}}" 
    # msg += ' \\\ '
    # msg += f"(\SIrange{{{x['lb']/1000:3.1f}}}{{{x['ub']/1000:3.1f}}}{{\{'micro'}\{'second'}}})"
    msg = f"\makecell[tc]{{{x['median']/1000:3.3f} \\\ ({x['lb']/1000:3.3f}, {x['ub']/1000:3.3f})}}"
    return msg
    

In [27]:
print(data.query('hp_ix==74').apply(label, axis=1).values[0])

\makecell[tc]{0.379 \\ (0.296, 0.470)}


In [20]:
data

,hp_ix,lag,process,median,lb,ub,count,cluster__max_iter,cluster__stride,tica__dim,...,tica__stride,cluster__k,feature__value,dihedrals__which,distances__scheme,distances__transform,distances__steepness,distances__centre,feature,rank
236,74,31,2,379.427824,296.450766,469.664699,100,1000,10,15,...,1,488,distances,NaN,ca,linear,0.000000,0.000000,dist.,1
4296,85,31,2,378.901482,294.835108,470.007656,100,1000,10,19,...,1,485,distances,NaN,ca,linear,0.000000,0.000000,dist.,2
981,131,31,2,377.822741,295.686109,467.770069,100,1000,10,18,...,1,380,distances,NaN,ca,linear,0.000000,0.000000,dist.,3
3003,24,31,2,377.699822,294.455989,466.445466,100,1000,10,20,...,1,471,distances,NaN,ca,logistic,6.081760,0.224891,logit(dist.),4
3200,60,31,2,376.557178,291.756685,465.965134,100,1000,10,11,...,1,285,distances,NaN,ca,linear,0.000000,0.000000,dist.,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,183,31,2,109.627366,100.357366,118.676078,100,1000,10,8,...,1,328,distances,NaN,ca,logistic,46.678902,1.458448,logit(dist.),127
3558,43,31,2,86.732057,78.654107,92.813997,100,1000,10,10,...,1,38,distances,NaN,closest-heavy,logistic,43.637496,1.232005,logit(dist.),128
3403,135,31,2,85.754788,79.888367,91.574037,100,1000,10,15,...,1,411,distances,NaN,closest-heavy,logistic,16.686443,1.491501,logit(dist.),129
626,22,31,2,69.385336,64.041383,74.553986,100,1000,10,6,...,1,148,distances,NaN,closest-heavy,logistic,33.504009,1.350308,logit(dist.),130
